# Preprocessing existing numpy file

In [ ]:
import pandas as pd
import numpy as np
df=pd.DataFrame(np.load('/home/vedant/projects/ScreamDetection/resources/working_data/data_with_vggish.npy',allow_pickle=True),columns=['video_id','start_time','mid_ts','label','audio','vggish'])
 
vgg=pd.DataFrame(df["vggish"].to_list())
cols=[]
for col in vgg.columns:
    cols.append(f"vggish_{col+1}")
vgg.columns= cols
 
df=pd.concat([df, vgg], axis=1)
df=df.drop('vggish',axis=1)
 
import soundfile as sf
v=''
i=0
df.insert(0, "block_number", '')
for index,row in df.iterrows():
    #print(row)
    video_id = row['video_id']
    if video_id == v:
        i+=1
    else:
        v=video_id
        i=0
    start_ts = row['start_time']
    df.iloc[index,0] = i
    out_name = f"{i}_{video_id}_start{int(start_ts*1000)}ms"
    print(out_name)
 
    audio = df.iloc[index,5]
    sf.write(audio,"/home/vedant/projects/ScreamDetection/resources/dataset/blocked_audio/"+out_name)

In [12]:
df=pd.read_csv('/home/vedant/projects/ScreamDetection/resources/dataset/dataset-pytorch.csv',header=0)

In [14]:
df['label'].unique()

array(['no_vocals', 'midfry', 'clean', 'highfry', 'lowfry', 'layered'],
      dtype=object)

In [22]:
df=pd.read_csv('/home/vedant/projects/ScreamDetection/resources/dataset/dataset-pytorch.csv',header=0)
#df.insert(6,'label_mapped','')
lst=[]
for index,row in df.iterrows():
    if row['label'] == 'no_vocals':
        lst.append(0)
    if row['label'] == 'midfry':
        lst.append(1)
    if row['label'] == 'clean':
        lst.append(2)
    if row['label'] == 'highfry':
        lst.append(3)
    if row['label'] == 'lowfry':
        lst.append(4)
    if row['label'] == 'layered':
        lst.append(5)
df.insert(6,'label_mapped',lst)

In [27]:
df

,block_number,video_id,start_time,mid_ts,label,filename,label_mapped,vggish_1,vggish_2,vggish_3,...,vggish_119,vggish_120,vggish_121,vggish_122,vggish_123,vggish_124,vggish_125,vggish_126,vggish_127,vggish_128
0,109,4600fGWcn9o,0.0,0.5,no_vocals,109_4600fGWcn9o_start0ms,0,185.0,16.0,145.0,...,0.0,228.0,127.0,255.0,92.0,125.0,90.0,98.0,6.0,255.0
1,110,4600fGWcn9o,0.5,1.0,no_vocals,110_4600fGWcn9o_start500ms,0,181.0,13.0,148.0,...,0.0,159.0,14.0,255.0,90.0,104.0,116.0,42.0,20.0,255.0
2,111,4600fGWcn9o,1.0,1.5,no_vocals,111_4600fGWcn9o_start1000ms,0,180.0,14.0,128.0,...,0.0,178.0,59.0,255.0,210.0,175.0,34.0,189.0,42.0,255.0
3,112,4600fGWcn9o,1.5,2.0,no_vocals,112_4600fGWcn9o_start1500ms,0,183.0,10.0,118.0,...,0.0,181.0,188.0,255.0,209.0,88.0,132.0,203.0,235.0,255.0
4,113,4600fGWcn9o,2.0,2.5,no_vocals,113_4600fGWcn9o_start2000ms,0,190.0,13.0,127.0,...,0.0,97.0,247.0,230.0,128.0,53.0,146.0,217.0,175.0,255.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33815,435,0m5fIHHfJTw,217.5,218.0,no_vocals,435_0m5fIHHfJTw_start217500ms,0,176.0,17.0,151.0,...,0.0,59.0,90.0,201.0,65.0,151.0,119.0,237.0,121.0,255.0
33816,436,0m5fIHHfJTw,218.0,218.5,no_vocals,436_0m5fIHHfJTw_start218000ms,0,174.0,15.0,143.0,...,108.0,195.0,197.0,60.0,117.0,21.0,255.0,106.0,0.0,255.0
33817,437,0m5fIHHfJTw,218.5,219.0,no_vocals,437_0m5fIHHfJTw_start218500ms,0,171.0,11.0,137.0,...,20.0,165.0,121.0,128.0,58.0,121.0,255.0,130.0,0.0,255.0
33818,438,0m5fIHHfJTw,219.0,219.5,no_vocals,438_0m5fIHHfJTw_start219000ms,0,174.0,12.0,138.0,...,0.0,138.0,136.0,186.0,48.0,139.0,233.0,36.0,0.0,255.0


In [24]:
df.to_csv('/home/vedant/projects/ScreamDetection/resources/dataset/dataset-pytorch.csv',header=True, index=False,encoding='utf-8-sig')

# Creating ScreamDataset Class

In [11]:
from torch.utils.data import Dataset
import torch
import pandas as pd
import torchaudio
import os
import numpy as np

In [ ]:
class UrbanSoundDataset(Dataset):
    def __init__(self, annotation_file,audio_dir,transformation,
                target_sample_rate,num_samples,device):
        self.annotations = pd.read_csv(annotation_file)
        self.audio_dir = audio_dir
        self.device = device
        self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples

    def __len__(self): #return number of elements in dataset
        return len(self.annotations)

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir,fold,self.annotations.iloc[index, 0 ])
        return path
    
    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]

    def _resample_if_necessary(self,signal,sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr,self.target_sample_rate).to(self.device)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self,signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal,dim=0,keepdim=True)
        return signal
    
    def _right_pad_if_necessary(self,signal):
        length_of_signal = signal.shape[1]
        if length_of_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_of_signal
            last_dim_padding = (0,num_missing_samples) # (left_pad,right_pad)
            signal = torch.nn.functional.pad(signal,last_dim_padding)
        return signal

    def _cut_if_necessary(self,signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:,:self.num_samples]
        return signal

    def __getitem__(self,index): #return item at an index
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device) #register signal to device
        #Transformations
        signal = self._resample_if_necessary(signal, sr) # convert to target sample rate
        signal = self._mix_down_if_necessary(signal) # converting to mono
        signal = self._right_pad_if_necessary(signal) # if num samples < target num samples
        signal = self._cut_if_necessary(signal) # crop if num samples > target num samples
        signal = self.transformation(signal) # Get mel spectrogram
        return signal, label



In [39]:
class ScreamDataset(Dataset):
    def __init__(self,annotation_file, audio_dir,transformation,device):
        self.annotations = pd.read_csv(annotation_file,header=0)
        self.audio_dir = audio_dir
        self.device = device
        self.transformation = transformation.to(self.device)

    def _get_audio_sample_path(self, index):
        filename = f"{self.annotations.iloc[index, 5]}.wav"
        path = os.path.join(self.audio_dir,filename)
        return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]

    def __len__(self):
        return len(self.annotations)
 

    def __getitem__(self,index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device) #register signal to device
        #Transformations
        signal = self.transformation(signal)
        return signal, label
 


In [8]:
annotation_file= '/home/vedant/projects/ScreamDetection/resources/dataset/dataset-pytorch.csv'

audio_dir= '/home/vedant/projects/ScreamDetection/resources/dataset/blocked_audio'

mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate = 44100,
        n_fft = 1024,
        hop_length = 512,
        n_mels = 64
    )

if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'


sd = ScreamDataset(annotation_file, audio_dir, mel_spectrogram, DEVICE)

In [38]:
sd[0]

(tensor([[[0.0000e+00, 0.0000e+00, 9.9935e-13,  ..., 4.5554e+02,
           4.5855e+02, 7.8074e+02],
          [0.0000e+00, 0.0000e+00, 1.0615e-12,  ..., 1.7162e+02,
           4.5055e+02, 2.5639e+02],
          [0.0000e+00, 0.0000e+00, 1.1111e-12,  ..., 2.7601e+01,
           2.2501e+02, 3.6884e+03],
          ...,
          [0.0000e+00, 0.0000e+00, 1.7160e-12,  ..., 9.1529e-07,
           6.5009e-06, 5.1676e-02],
          [0.0000e+00, 0.0000e+00, 1.2231e-12,  ..., 1.2752e-06,
           4.0309e-06, 5.2595e-02],
          [0.0000e+00, 0.0000e+00, 1.2521e-12,  ..., 1.2896e-06,
           2.1250e-06, 5.4387e-02]]], device='cuda:0'),
 'no_vocals')

In [36]:
from screamdataset import ScreamDataset
import torch 
import torchaudio
from torch import nn
from torch.utils.data import DataLoader
from cnn import CNNNetwork

ANNOTATIONS_FILE = '/home/vedant/projects/ScreamDetection/resources/dataset/dataset-pytorch.csv'
AUDIO_DIR = '/home/vedant/projects/ScreamDetection/resources/dataset/blocked_audio'
BATCH_SIZE = 128
SAMPLE_RATE=44100
EPOCHS = 30
LEARNING_RATE = 0.001

mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate = 44100,
        n_fft = 1024,
        hop_length = 512,
        n_mels = 64
    )
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

def create_data_loader(train_data,batch_size):
    train_dataloader = DataLoader(train_data,batch_size=batch_size)
    return train_dataloader

def train_one_epoch(model, data_loader,loss_function,optimiser,device):
    for inputs,targets in data_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        #inputs,targets = inputs.to(device),targets.to(device)

        # Calculate Loss
        predictions = model(inputs)
        loss = loss_function(predictions,targets)

        # Backpropagate Loss, update weights
        optimiser.zero_grad()
        loss.backward() # Apply backpropagation
        optimiser.step() # Update weights
    print(f"Loss : {loss.item()}")

def train(model, data_loader,loss_function, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}:")
        train_one_epoch(model, data_loader, loss_function, optimiser, device)
        print("-------------------------------------------------------")
    print("Training done")

In [30]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

print(f"Using device: {DEVICE}")



#instantiating dataset object and transform
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate = SAMPLE_RATE,
    n_fft = 1024,
    hop_length = 512,
    n_mels = 64
)

sd = ScreamDataset(ANNOTATIONS_FILE, AUDIO_DIR, mel_spectrogram, DEVICE)
train_dataloader = create_data_loader(sd,BATCH_SIZE)

cnn = CNNNetwork().to(DEVICE)
# Instantiating loss function and optimiser
loss_function = nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(cnn.parameters(),
                        lr=LEARNING_RATE
                            )

#Train Model
train(cnn,train_dataloader, loss_function, optimiser, DEVICE, EPOCHS)

#Save results
torch.save(cnn.state_dict(),"/home/vedant/projects/ScreamDetection/CNN/trained_models/scream_cnn.pth")

print("Model trained and stored at /CNN/trained_models/scream_cnn.pth")

Using device: cuda
Epoch 1:


AttributeError: 'tuple' object has no attribute 'to'

In [31]:
i=0
for inputs,targets in train_dataloader:
    if i==0:
        print(targets)
        break

('no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'no_vocals', 'midfry', 'midfry', 'midfry', 'midfry', 'midfry', 'midfry', 'midfry', 'midfr